<a href="https://colab.research.google.com/github/Yohk2291/MachineLearning/blob/master/ML/Titanic_aijc32test190706_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic

2019年7月6日実施のAIジョブカレ機械学習講座32期最終開発課題で作成したモデルを
Titanicで使用してみました。

修了認定のための最終開発課題

自分用の Automated Machine Learning を開発しよう! 

【機能要件】

・２値の分類(Classification)タスクを扱える

・カテゴリカル変数を指定すると one-hot エンコードを実行する

・モデル用データマートに施したのと同一データ前処理をスコア用データマートに対しても適用される

・モデル選択の評価指標を選択できる

・複数アルゴリズムから指定の評価指標に従いベストモデルを選択できる

・学習済みモデルを保存できる

・アルゴリズムランキングと性能評価指標が出力される


## colaboratory上でsubmitするかを選択

In [226]:
import ipywidgets as widgets

def get_answer(x):
    return x

submit = get_answer(widgets.RadioButtons(options=['yes', 'no']))
print('submitしますか？')
display(submit)

submitしますか？


RadioButtons(options=('yes', 'no'), value='yes')

In [227]:
print('submitしますか？：',submit.value)

submitしますか？： no


In [228]:
submission = {'yes':1, 'no':0}
kaggle_submit = submission[submit.value]
print(kaggle_submit)

0


## submitする場合の処理

In [0]:
from google.colab import files

if kaggle_submit == 1:
  files.upload()

# "Create New API Token"よりkaggle.jsonをダウンロードして任意のフォルダに保存
# 詳細は https://qiita.com/katsu1110/items/a8d508a1b6f07bd3a243 

In [0]:
if kaggle_submit == 1:
  !mkdir -p ~/.kaggle
  !mv kaggle.json ~/.kaggle/

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

if kaggle_submit == 1:
  auth.authenticate_user()

  drive_service = build('drive', 'v3')
  results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
  kaggle_api_key = results.get('files', [])

  filename = "/root/.kaggle/kaggle.json"
  os.makedirs(os.path.dirname(filename), exist_ok=True)

  request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
  fh = io.FileIO(filename, 'wb')
  downloader = MediaIoBaseDownload(fh, request)
  done = False
  while done is False:
      status, done = downloader.next_chunk()
      print("Download %d%%." % int(status.progress() * 100))
  os.chmod(filename, 600)


In [0]:
if kaggle_submit == 1:
  !pip install kaggle

  !kaggle competitions download -c titanic

In [233]:
!git clone https://github.com/Yohk2291/MachineLearning.git
%cd MachineLearning/
%cd ML/

Cloning into 'MachineLearning'...
remote: Enumerating objects: 110, done.
remote: Total 110 (delta 0), reused 0 (delta 0), pack-reused 110
Receiving objects: 100% (110/110), 1.99 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning
/content/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML/MachineLearning/ML


## 各種ライブラリ、パッケージのインポート

In [0]:
# ライブラリのimport
from IPython.core.display import display
import numpy as np
import pandas as pd
import pandas_profiling as pdp

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,f1_score, roc_auc_score

from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler,SMOTE

import warnings
warnings.filterwarnings('ignore')


## csvの読み込み

In [235]:
# データ型を指定して読み込み
my_dtype = {'Name':object,
            'Sex':object,
            'Embarked':object}


df = pd.read_csv('./data/train.csv', header=0, dtype=my_dtype)
df_s = pd.read_csv('./data/test.csv', header=0, dtype=my_dtype)
sample_submission = pd.read_csv('./data/gender_submission.csv')

pdp.ProfileReport(df)

Number of variables,12
Number of observations,891
Total Missing (%),8.1%
Total size in memory,83.6 KiB
Average record size in memory,96.1 B
Numeric,6
Categorical,4
Boolean,1
Date,0
Text (Unique),1
Rejected,0


In [236]:
X = df.iloc[:,2:] # PassengerId, Survivedを除外(ID, y)
X = X.drop(['Ticket','Cabin'], axis=1) # Ticket, Cabinのデータは今回使用しない
ID = df.iloc[:,[0]]
y = df.iloc[:,1]


# check the shape
print('--------------------------------------')
print('Raw shape: (%i,%i)' %df.shape)
print('X shape: (%i,%i)' %X.shape)

print('---------------------------------------')
print(y.value_counts())
print('---------------------------------------')
print(ID.join(X).join(y).dtypes)

# 表示列数のオプション変更
pd.options.display.max_columns = 50

print(X.isnull().sum())

--------------------------------------
Raw shape: (891,12)
X shape: (891,8)
---------------------------------------
0    549
1    342
Name: Survived, dtype: int64
---------------------------------------
PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
Survived         int64
dtype: object
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64


## Trainデータの前処理

In [237]:
#名前の敬称抽出
X['Name'] = X.Name.str.extract('([A-Za-z]+)\.', expand=False)
X.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,Mr,male,22.0,1,0,7.2500,S
1,1,Mrs,female,38.0,1,0,71.2833,C
2,3,Miss,female,26.0,0,0,7.9250,S
3,1,Mrs,female,35.0,1,0,53.1000,S
4,3,Mr,male,35.0,0,0,8.0500,S


In [238]:
#カテゴリ変数（'Name','Sex','Embarked'）をone-hot-encoding（Nanも１カテゴリとする）
ohe_columns = ['Name','Sex','Embarked']
X_ohe = pd.get_dummies(X,dummy_na=True,columns=ohe_columns)

print('X_ohe.shape:(%i,%i)'%X_ohe.shape)
display(X_ohe.head())

X_ohe.shape:(891,30)


,Pclass,Age,SibSp,Parch,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3,22.0,1,0,7.2500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
1,1,38.0,1,0,71.2833,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
2,3,26.0,0,0,7.9250,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,1,35.0,1,0,53.1000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
4,3,35.0,0,0,8.0500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0


In [239]:
#欠損値Nanを平均値で補完
imp = SimpleImputer()
imp.fit(X_ohe)

X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

display(X_ohe.head())

,Pclass,Age,SibSp,Parch,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3.0,22.0,1.0,0.0,7.2500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,38.0,1.0,0.0,71.2833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3.0,26.0,0.0,0.0,7.9250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3.0,35.0,0.0,0.0,8.0500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [240]:
selector = RFE(RandomForestClassifier(n_estimators=100,random_state=1),n_features_to_select=10,step=.05)

selector.fit(X_ohe,y)

X_fin = pd.DataFrame(selector.transform(X_ohe),columns=X_ohe_columns[selector.support_])

print('X_fin shape:(%i,%i)'%X_fin.shape)
display(X_fin.head())

X_fin shape:(891,10)


,Pclass,Age,SibSp,Parch,Fare,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male
0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1.0,38.0,1.0,0.0,71.2833,0.0,0.0,1.0,1.0,0.0
2,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,1.0,0.0
3,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,1.0,1.0,0.0
4,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0


## Testデータの前処理

In [241]:
ID_s = df_s.iloc[:,[0]]
X_s = df_s.iloc[:,1:]
X_s = X_s.drop(['Ticket','Cabin'], axis=1)

print('Raw Shape:(%i,%i)'%df_s.shape)
print('X_shape:(%i,%i)'%X_s.shape)
print('-------------------------------')
print(X_s.dtypes)

Raw Shape:(418,11)
X_shape:(418,8)
-------------------------------
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object


In [242]:
#名前の敬称抽出
X_s['Name'] = X_s.Name.str.extract('([A-Za-z]+)\.', expand=False)
X_s.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,Mr,male,34.5,0,0,7.8292,Q
1,3,Mrs,female,47.0,1,0,7.0000,S
2,2,Mr,male,62.0,0,0,9.6875,Q
3,3,Mr,male,27.0,0,0,8.6625,S
4,3,Mrs,female,22.0,1,1,12.2875,S


In [243]:
X_ohe_s = pd.get_dummies(X_s,dummy_na=True,columns=ohe_columns)
print('X_ohe_s shape:(%i,%i)'%X_ohe_s.shape)
display(X_ohe_s.head())

X_ohe_s shape:(418,22)


,Pclass,Age,SibSp,Parch,Fare,Name_Col,Name_Dona,Name_Dr,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3,34.5,0,0,7.8292,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1,3,47.0,1,0,7.0000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
2,2,62.0,0,0,9.6875,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
3,3,27.0,0,0,8.6625,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
4,3,22.0,1,1,12.2875,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0


In [244]:
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

diff1 = cols_model - cols_score
print('Modelのみ:%s'%diff1)

diff2 = cols_score - cols_model
print('Scoreのみ:%s'%diff2)

Modelのみ:{'Name_Mme', 'Name_Sir', 'Name_Lady', 'Name_Mlle', 'Name_Jonkheer', 'Name_Major', 'Name_Capt', 'Name_Countess', 'Name_Don'}
Scoreのみ:{'Name_Dona'}


In [245]:
df_cols_m = pd.DataFrame(None,
                         columns=X_ohe_columns,
                         dtype=float)
display(df_cols_m)

,Pclass,Age,SibSp,Parch,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan


In [246]:
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
print(X_ohe_s2.shape)
display(X_ohe_s2.head(3))

(418, 31)


,Age,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dona,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Parch,Pclass,Sex_female,Sex_male,Sex_nan,SibSp
0,34.5,0.0,1.0,0.0,0.0,7.8292,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,3.0,0.0,1.0,0.0,0.0
1,47.0,0.0,0.0,1.0,0.0,7.0000,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,0.0,0.0,3.0,1.0,0.0,0.0,1.0
2,62.0,0.0,1.0,0.0,0.0,9.6875,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,2.0,0.0,1.0,0.0,0.0


In [247]:
set_Xm = set(X_ohe.columns.values)
set_Xs = set(X_ohe_s.columns.values)

X_ohe_s3 = X_ohe_s2.drop(list(set_Xs-set_Xm),axis=1)

print(X_ohe_s3.shape)
display(X_ohe_s3.head(3))

(418, 30)


,Age,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Parch,Pclass,Sex_female,Sex_male,Sex_nan,SibSp
0,34.5,0.0,1.0,0.0,0.0,7.8292,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,3.0,0.0,1.0,0.0,0.0
1,47.0,0.0,0.0,1.0,0.0,7.0000,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,0.0,0.0,3.0,1.0,0.0,0.0,1.0
2,62.0,0.0,1.0,0.0,0.0,9.6875,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0,2.0,0.0,1.0,0.0,0.0


In [248]:
X_ohe_s3.loc[:,list(set_Xm-set_Xs)] = X_ohe_s3.loc[:,list(set_Xm-set_Xs)].fillna(0,axis=1)
X_ohe_s3.head(3)

,Age,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Parch,Pclass,Sex_female,Sex_male,Sex_nan,SibSp
0,34.5,0.0,1.0,0.0,0.0,7.8292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0
1,47.0,0.0,0.0,1.0,0.0,7.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0
2,62.0,0.0,1.0,0.0,0.0,9.6875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0


In [249]:
X_ohe_s3 = X_ohe_s3.reindex(X_ohe.columns.values,axis=1)
X_ohe_s3.head(3)

,Pclass,Age,SibSp,Parch,Fare,Name_Capt,Name_Col,Name_Countess,Name_Don,Name_Dr,Name_Jonkheer,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Ms,Name_Rev,Name_Sir,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,3.0,34.5,0.0,0.0,7.8292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,3.0,47.0,1.0,0.0,7.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,62.0,0.0,0.0,9.6875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [250]:
print('欠損個数（数値変数の欠損補完前）',X_ohe_s3.isnull().sum().sum())
X_ohe_s4 = pd.DataFrame(imp.transform(X_ohe_s3),columns=X_ohe_columns)
print('欠損個数（数値変数の欠損補完後）',X_ohe_s4.isnull().sum().sum())

欠損個数（数値変数の欠損補完前） 87
欠損個数（数値変数の欠損補完後） 0


In [251]:
X_fin_s = X_ohe_s4.loc[:, X_ohe_columns[selector.support_]]
print(X_fin_s.shape)
X_fin_s.head(20)

(418, 10)


,Pclass,Age,SibSp,Parch,Fare,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male
0,3.0,34.500000,0.0,0.0,7.8292,0.0,1.0,0.0,0.0,1.0
1,3.0,47.000000,1.0,0.0,7.0000,0.0,0.0,1.0,1.0,0.0
2,2.0,62.000000,0.0,0.0,9.6875,0.0,1.0,0.0,0.0,1.0
3,3.0,27.000000,0.0,0.0,8.6625,0.0,1.0,0.0,0.0,1.0
4,3.0,22.000000,1.0,1.0,12.2875,0.0,0.0,1.0,1.0,0.0
5,3.0,14.000000,0.0,0.0,9.2250,0.0,1.0,0.0,0.0,1.0
6,3.0,30.000000,0.0,0.0,7.6292,1.0,0.0,0.0,1.0,0.0
7,2.0,26.000000,1.0,1.0,29.0000,0.0,1.0,0.0,0.0,1.0
8,3.0,18.000000,0.0,0.0,7.2292,0.0,0.0,1.0,1.0,0.0
9,3.0,21.000000,2.0,0.0,24.1500,0.0,1.0,0.0,0.0,1.0


## モデルの構築

In [0]:
pipelines={
    'knn':Pipeline([('scl', StandardScaler()), ('est', KNeighborsClassifier())]),
    'logistic':Pipeline([('scl', StandardScaler()), ('est', LogisticRegression(random_state=1))]),
    'rsvc':Pipeline([('scl', StandardScaler()), ('est', SVC(random_state=1))]),
    'lsvc':Pipeline([('scl',StandardScaler()),('est',LinearSVC(C=1.0, class_weight='balanced', random_state=1))]),
    'tree':Pipeline([('scl', StandardScaler()), ('est', DecisionTreeClassifier(random_state=1))]),
    'rf':Pipeline([('scl', StandardScaler()), ('est', RandomForestClassifier(random_state=1))]),
    'gb':Pipeline([('scl', StandardScaler()), ('est', GradientBoostingClassifier(random_state=1))]),
    'lgb':Pipeline([('scl', StandardScaler()), ('est', LGBMClassifier(random_state=1))]),
    'mlp':Pipeline([('scl', StandardScaler()), ('est', MLPClassifier(hidden_layer_sizes=(3,3),max_iter=1000))])
}


In [0]:
# パラメータの候補
param_grid = {'knn':{'est__n_neighbors':[1,2,3,4,5]},
              'logistic':{'est__C':np.arange(2,4,1)},
              'rsvc':{'est__C':np.arange(0.1,1,0.05)},
              'lsvc':{'est__C':np.arange(4,7,1)},
              'tree':{'est__max_depth': [7,8,9]},
              'rf':{'est__max_depth': [2,3,6,None],
                    'est__n_estimators':[200],
                    'est__max_features': [3,4,5]},
              'gb':{'est__learning_rate':[0.01,0.05,0.1],
                    'est__n_estimators':[100],
                    'est__max_depth':[5,6,7]},
              'lgb':{'est__learning_rate': [0.15,0.20,0.25],
                     'est__num_iterations':[100,200,300],
                     'est__n_estimators': [40],
                     'est__num_leaves': list(range(16, 33, 2))},
              'mlp':{'est__learning_rate': ['constant', 'adaptive'],
                     'est__solver':['sgd', 'adam'],
                     'est__activation': ['logistic', 'tanh', 'relu']}
             }

## 評価指標選択(accuracy, f1, roc_aucから選択可)

In [254]:
import ipywidgets as widgets

def get_answer(x):
    return x

evaluation_choice_grid = get_answer(widgets.RadioButtons(options=['accuracy', 'f1', 'roc_auc']))
display(evaluation_choice_grid)

RadioButtons(options=('accuracy', 'f1', 'roc_auc'), value='accuracy')

In [255]:
print('選択した評価指標：',evaluation_choice_grid.value)

選択した評価指標： accuracy


In [256]:
from sklearn.model_selection import GridSearchCV
scores={}
pipelines_gs = {}

# pipelinesの各モデルを順に実行
for pipe_name, pipeline in pipelines.items():
    print(pipe_name + ' Start')
    pipeline_gs = GridSearchCV(estimator=pipeline,
                  param_grid=param_grid[pipe_name],
                  scoring=evaluation_choice_grid.value,
                  cv=3,
                  return_train_score=False)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    
    train_accuracy=[]
    test_accuracy=[]
    train_f1=[]
    test_f1=[]
    train_auc=[]
    test_auc=[]
    
    # X_fin, yを KFoldで分割し、予測してscoreを算出
    for fold_, (train_index, test_index) in enumerate(kf.split(X_fin, y)):
        X_train = X_fin.iloc[train_index]
        X_test = X_fin.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        
        print(' fold{0} '.format(fold_ + 1))
        pipeline_gs.fit(X_train, y_train)
        
        train_accuracy.append(accuracy_score(y_train, pipeline_gs.predict(X_train)))
        test_accuracy.append(accuracy_score(y_test, pipeline_gs.predict(X_test)))
        train_f1.append(f1_score(y_train, pipeline_gs.predict(X_train)))
        test_f1.append(f1_score(y_test, pipeline_gs.predict(X_test)))
        train_auc.append(roc_auc_score(y_train, pipeline_gs.predict(X_train)))
        test_auc.append(roc_auc_score(y_test, pipeline_gs.predict(X_test)))
    
    
    # scoreの平均値を最終的な予測スコアとして使用　
    scores[(pipe_name, 'train_accuracy')] = np.mean(train_accuracy)
    scores[(pipe_name, 'test_accuracy')] = np.mean(test_accuracy)
    scores[(pipe_name, 'train_f1')] = np.mean(train_f1)
    scores[(pipe_name, 'test_f1')] = np.mean(test_f1)
    scores[(pipe_name, 'train_auc')] = np.mean(train_auc)
    scores[(pipe_name, 'test_auc')] = np.mean(test_auc)
    
    
    pipelines_gs[pipe_name] = pipeline_gs.best_estimator_
    
    print(pipe_name + ' End')
    print('Best Params:{}'.format(pipeline_gs.best_params_))
    print('-'*100)

knn Start
 fold1 
 fold2 
 fold3 
 fold4 
 fold5 
knn End
Best Params:{'est__n_neighbors': 4}
----------------------------------------------------------------------------------------------------
logistic Start
 fold1 
 fold2 
 fold3 
 fold4 
 fold5 
logistic End
Best Params:{'est__C': 3}
----------------------------------------------------------------------------------------------------
rsvc Start
 fold1 
 fold2 
 fold3 
 fold4 
 fold5 
rsvc End
Best Params:{'est__C': 0.40000000000000013}
----------------------------------------------------------------------------------------------------
lsvc Start
 fold1 
 fold2 
 fold3 
 fold4 
 fold5 
lsvc End
Best Params:{'est__C': 6}
----------------------------------------------------------------------------------------------------
tree Start
 fold1 
 fold2 
 fold3 
 fold4 
 fold5 
tree End
Best Params:{'est__max_depth': 8}
----------------------------------------------------------------------------------------------------
rf Start
 fold1 
 fold2

## 性能評価指標を選択(GridSearchの評価指標と連動)

In [0]:
evaluation_choice = {'accuracy':'test_accuracy', 'f1':'test_f1', 'roc_auc':'test_auc'}

## モデルのランキング

In [258]:
#選択した評価指標でソート
scores_ranking = pd.Series(scores).unstack().sort_values(evaluation_choice[evaluation_choice_grid.value], ascending=False)
display(scores_ranking)

,test_accuracy,test_auc,test_f1,train_accuracy,train_auc,train_f1
rsvc,0.828291,0.811635,0.764127,0.834736,0.816726,0.774324
rf,0.818191,0.802991,0.752682,0.892252,0.877043,0.851624
lgb,0.813671,0.801650,0.751305,0.957628,0.950595,0.943195
gb,0.809227,0.792698,0.735740,0.898984,0.882501,0.859377
knn,0.809190,0.789940,0.734282,0.867565,0.849999,0.818038
logistic,0.806980,0.790472,0.737929,0.815659,0.799167,0.751954
tree,0.801362,0.788787,0.733146,0.898714,0.886105,0.863126
mlp,0.799115,0.775938,0.714828,0.819027,0.795192,0.745987
lsvc,0.797998,0.790134,0.738314,0.813414,0.804726,0.759321


## ベストモデル・スコア、予測確率の表示

In [259]:
best_model = scores_ranking.index[0]
print('-'*80)
print('BestModel:',best_model)
print('-'*80)

clf = pipelines_gs[best_model]
print(clf)


--------------------------------------------------------------------------------
BestModel: rsvc
--------------------------------------------------------------------------------
Pipeline(memory=None,
         steps=[('scl',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('est',
                 SVC(C=0.40000000000000013, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)


## 学習済モデルの保存

In [0]:
import pickle

s = pickle.dumps(clf)
clf2 = pickle.loads(s)

#prediction2 = clf2.predict(X_fin_s)

## 提出用ファイルへの書き出し

In [0]:
test_pred = clf.predict(X_fin_s)

sample_submission['Survived'] = test_pred
sample_submission.to_csv('submission.csv',index=False)

In [0]:
# colaboratory上でsubmitする場合に使用
if kaggle_submit == 1:
  !kaggle competitions submit -c titanic -f submission.csv -m "Yeah! I submit my file through the Google Colab!"
